### Stranger Sections 2 - Helper Notebook

Thank you for participating the the Stranger Sections 2 Challenge! This challenge aims for participants to build a model that is capable of segmenting kerogen macerals on reflected light polished sections. The goal of this helper notebook is to spark ideas for unique ways to solve this challenge. You do not need to be a geoscientist to be successful in this challenge. In fact, unique approaches from other disciplines might be just what we need to solve the problem! Please familiarize yourself with the challenge description, evaluation criteria, and data size and type before diving into this helper notebook. You can find all relevant information on the [challenge page](https://thinkonward.com/app/c/challenges/stranger-sections-2).

**You did what?!** For this challenge we realeased a LOT of data. 8,728 images of reflected light polished sections to be exact. Of these images, 87 have segmentation labels. Given that most of the data is unlabeled, what are some creative ways to use this data. We don't assume or expect challengers to be organic petrologists, so labeling by hand seems out of the question. One method the Onward team has been exploring is Vision Transformers! What happens if we further pretrain an existing ViT backbone with unlabeled thin section images?! Can we use our fine-tuning module to get improved segmentation results over an out of the box method?

#### Backbone Pre-training

In the starter notebook we provided code that gives you the ability to further pre-train a Vision Transformer backbone using the unlabeled images from the challenge. The pre-training code can be found in the `masked_autoencoder.pretrain` module. The pretraining script can be run in Jupyter or terminal. Please see the the starter notebook for instructions to run in terminal.

In [ ]:
import sys

sys.path.append("masked_autoencoder")
from masked_autoencoder.pretrain import pretrain_mae
from utils import plot_report

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np

In [ ]:
# Data in your dataset catalog should contain 3D numpy arrays stored as `.npy` files. We recommend to start with data from Stranger Sections 2 challange.
dataset = "./data/unlabeled_images/"

# We will start with default torchvision checkpoint - pretrained on ImageNet1k data
history = pretrain_mae(
    dataset,  # your checkpoint will be saved to this location - catalog must exist before training!
    output="checkpoints/ViT_B_16_stranger_sections2.pth",
    transform_kwargs={
        "min_scale": 0.33,
        "normalize": False,
    },  # this will be passed to lightly.transforms.mae_transform.MAETransform
    vit_model="ViT_B_16",
    starting_weights="ViT_B_16_Weights.DEFAULT",
    batch_size=64,
    n_workers=8,  # to achieve best performance we recommend setting this value to match number of physical cores of your CPU
    optimizer="SGD",  # AdamW and SGD are supported out of the box
    optimizer_kwargs={
        "lr": 1e-4,
        "momentum": 0.0,
    },  # this will be passed to optimizer constructor
    warmup_epochs=20,  # tune Learning Rate schedules to match your needs
    start_factor=0.2,  # starting decrease factor for warmup
    linear_schedule=True,
    end_factor=0.5,  # final decrease factor for LR schedule
    n_epochs=200,  # number of non-warmup training epochs
    masking_rate=0.75,  # define what share of input image will be masked
    decoder_dim=256,  # we will use smaller decoder as we're training the smallest model of ViT family
    freeze_projection=False, # we will start with trainig whole model
    freeze_embeddings=False,
)  

In [ ]:
plot_report(history, x_locator_tick=2)

#### Segmentation Fine-tuneing

In [ ]:
# Fine-tuning code here